In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from mnist_example import MNISTModel, MNISTDataModule

datamodule = MNISTDataModule(batch_size=10)
datamodule.setup()

len(datamodule.test_dataloader())

10

In [3]:
from energizer.data.datamodule import ActiveDataModule

dm = ActiveDataModule(datamodule=datamodule)
dm.setup()
dm.setup_folds()

dm.is_on_pool = False
print(len(dm.test_dataloader()))
dm.is_on_pool = True
print(len(dm.test_dataloader()))

10
10


In [4]:
# print(
#     len(dm.pool_fold), 
#     len(dm.test_dataloader()),
#     len(dm.train_fold), 
#     len(dm.train_dataloader()),
# )
# dm.label(list(range(100)))
# print(
#     len(dm.pool_fold), 
#     len(dm.test_dataloader()),
#     len(dm.train_fold), 
#     len(dm.train_dataloader()),
# )

---

In [5]:
import torch
from torch.utils.data import DataLoader
from energizer.learners.base import Deterministic
from energizer.learners.acquisition_functions import entropy
from pytorch_lightning import LightningModule, Trainer as pl_Trainer
from energizer.trainer import Trainer

In [6]:
class Learner(Deterministic):

    def pool_step(self, batch, *args, **kwargs) -> torch.Tensor:
        x, _ = batch
        logits = self(x)
        scores = entropy(logits)
        return scores

In [7]:
model = MNISTModel()
active_learner = Learner(model)

In [8]:
pl_trainer = pl_Trainer(max_epochs=1)
pl_trainer.fit(active_learner, datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params
---------------------------------------
0 | learner | MNISTModel | 184 K 
---------------------------------------
184 K     Trainable params
0         Non-trainable params
184 K     Total params
0.738     Total estimated model params size (MB)


/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:589: UserWarning: You have overridden `on_before_batch_transfer` in `LightningModule` but have passed in a `LightningDataModule`. It will use the implementation from `LightningModule` instance.
  f"You have overridden `{hook_name}` in `LightningModule` but have passed in a"
/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:589: UserWarning: Yo

Epoch 0: 100%|██████████| 20/20 [00:00<00:00, 67.84it/s, loss=2.32, v_num=314]


In [9]:
pl_trainer.test(active_learner, datamodule)

/Users/pietrolesci/miniconda3/envs/energizer-dev/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Testing DataLoader 0: 100%|██████████| 10/10 [00:00<00:00, 157.34it/s]


[{}]

In [10]:
trainer = Trainer(query_size=2, max_epochs=1)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [11]:
trainer.fit(model, datamodule)


  | Name  | Type             | Params
-------------------------------------------
0 | model | Sequential       | 184 K 
1 | loss  | CrossEntropyLoss | 0     
-------------------------------------------
184 K     Trainable params
0         Non-trainable params
184 K     Total params
0.738     Total estimated model params size (MB)


Epoch 0: 100%|██████████| 20/20 [00:00<00:00, 61.98it/s, loss=2.3, v_num=315] 


In [13]:
trainer.test(model, datamodule)

Testing:   0%|          | 0/10 [01:35<?, ?it/s]
Testing: 10it [00:00, 113.19it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test/loss         │    2.2414803504943848     │
└───────────────────────────┴───────────────────────────┘

[{'test/loss': 2.2414803504943848}]

In [ ]:
test_dl = DataLoader(
    [datamodule.mnist_test[i] for i in range(10)], 
    shuffle=False, 
    batch_size=2,
)
x, y = next(iter(test_dl))

In [ ]:
model = MNISTModel()
adapter = MCDropout(model)
strategy = EntropyStrategy(adapter, 3)
dm = ActiveDataModule(datamodule=datamodule)
dm.setup()
dm.setup_folds()

In [ ]:
len(dm.pool_fold), len(dm.train_fold)

In [ ]:
datamodule.train_dataloader().batch_size

In [ ]:
trainer = Trainer(max_epochs=1, enable_progress_bar=True, total_budget=3)
trainer.active_fit(strategy, datamodule=dm)

In [ ]:
len(dm.val_dataloader())

In [ ]:
trainer.fit_loop.done

In [ ]:
trainer.fit(strategy, datamodule=datamodule)

In [ ]:
trainer.current_epoch